# Import dependencies

In [ ]:
import nltk
from os import listdir
import time
from nltk.corpus import wordnet
import re
import pickle

# Config & Hyper Parameters

In [ ]:
base_path = 'G:\\AI\\data\\cnn\\'
path = base_path + 'sample_5k\\'
articles_pickle_filename = "articles.pickle"
headlines_pickle_filename = "headlines.pickle"
model_pickle_filename = "model.pickle"
word_embedding_matrix_filename = "word_embedding_matrix.pickle"

''' https://fasttext.cc/docs/en/english-vectors.html '''
model_path ='G:\Python\MLLearning\MachineLearning\data\wiki-news-300d-1M.vec'

# to avoid words that are used less that threshold value
threshold = 2

# Dimension size as per pre-trained data
embedding_dim = 300
max_text_length = 1000
max_summary_length = 20
min_length = 2
unk_text_limit = 200

# Set the Hyperparameters
epochs = 100
batch_size = 64
rnn_size = 256
num_layers = 2
learning_rate = 0.005
keep_probability = 0.75

# Stopword list and Initialize Lemmatizer

In [ ]:
nltk.download('stopwords')

stop_words = nltk.corpus.stopwords.words('english')
lmtzr = nltk.WordNetLemmatizer().lemmatize


# Read files and load into memory

In [ ]:
def load_files(filename):
    # open the file as read only
    file = open(filename, encoding='utf-8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

# split a document into news article body and headlines

In [ ]:
def split_data(doc):
    # find first headlines
    index = doc.find('@highlight')
    # split into story and headlines
    article, headlines = doc[:index], doc[index:].split('@highlight')
    # strip extra white space around each highlight
    headlines = [h.strip() for h in headlines if len(h) > 0]
    return article, headlines

# Clean a list of lines
This section is used to remove unwanted words and return cleaned articles and headlines.

In [ ]:
def clean_text(lines, remove_stopwords=True):
    
    cleaned = list()
    for line in lines:
        # strip source cnn office if it exists
        index = line.find('(CNN)  -- ')
        if index > -1:
            line = line[index + len('(CNN)'):]
        else:
            index = line.find('(CNN)')
            if index > -1:
                line = line[index + len('(CNN)'):]

        # tokenize on white space
        line = line.split()

        # convert to lower case
        line = [word.lower() for word in line]

        # Optionally, remove stop words
        if remove_stopwords:
            line = [w for w in line if w not in stop_words]

        # remove punctuation from each token
        #line = [w.translate(table) for w in line]

        # remove tokens with numbers in them
        line = [word for word in line if word.isalpha()]

        # Format words and remove unwanted characters
        text = " ".join(line)
        text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
        text = re.sub(r'\<a href', ' ', text)
        text = re.sub(r'&amp;', '', text)
        text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
        text = re.sub(r'<br />', ' ', text)
        text = re.sub(r'\'', ' ', text)

        # remove empty strings
        if len(text )> 0 :
            cleaned.append(text)

    return cleaned

# Normalization of data using Lemmatization
Lemmatization is used as it returns better words choice than stemming as Lemmatization returns only valid dictionary words. Trade is it takes more time.

In [ ]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def normalize_text(text):
    cleaned = list()

    for line in text :
        word_pos = nltk.pos_tag(nltk.word_tokenize(line))
        lemm_words = [lmtzr(sw[0], get_wordnet_pos(sw[1])) for sw in word_pos]

        word = [x.lower() for x in lemm_words]
        cleaned.append(' '.join(word))

    return cleaned

# load all stories in a directory
This is used to load and clean the learn and test dataset. After cleaning data it returns two list cleaned articles and cleaned headlines.

In [ ]:
def load_stories(location):
    stories = list()
    file_list = listdir(location)
    total_files = len (file_list)
    count = 0
    print ("Total Files : {total_files}".format(total_files= total_files))
    clean_articles = []
    clean_headlines = []
    for name in file_list:
        count = count + 1
        filename = location + '/' + name
        # load document
        print('Loading  - {filename}, files number  - {count},  out of - {total_files}'
              .format(filename=filename, count=count, total_files=total_files))
        doc = load_files(filename)
        # split into story and highlights
        article, headlines = split_data(doc)
        # store
        #stories.append({'article': article, 'headlines' : headlines})

        article = clean_text(article.split('\n'))
        article = normalize_text(article)
        clean_articles.append(' '.join(article))
        headlines = clean_text(headlines, remove_stopwords=False)
        headlines = normalize_text(headlines)
        clean_headlines.append(' '.join(headlines))

    return clean_articles, clean_headlines

# Main Program
Start point of data cleaning, once the articles and headlines are cleaned, they dumped so that can be reused for vectorization and then running model directly. This is becasue cleaning is an expensive operation in terms of time and resources. 

In [12]:
def main():
    start = time.perf_counter()
    clean_articles, clean_headlines = load_stories(path)
    print("Total Articles  : {len_articles} , Total Headlines : {len_headlines}- Time Taken : {time_taken}"
          .format(len_articles=len(clean_articles), len_headlines =len(clean_headlines), time_taken = (time.perf_counter()-start)/60))

    print ("Serialization of articles")
    # Store Articles (serialize)
    with open(base_path + articles_pickle_filename, 'wb') as handle:
        pickle.dump(clean_articles, handle, protocol=pickle.HIGHEST_PROTOCOL)

    print("Serialization of headlines")
    # Store Articles (serialize)
    with open(base_path + headlines_pickle_filename, 'wb') as handle:
        pickle.dump(clean_headlines, handle, protocol=pickle.HIGHEST_PROTOCOL)


'''-------------------------main------------------------------'''
main()


Total Files : 5607
Loading  - G:\AI\data\cnn\sample_5k\/001ee59c375363263821474d40e4386ab91d5145.story, files number  - 1,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/00200e794fa41d3f7ce92cbf43e9fd4cd652bb09.story, files number  - 2,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/0021fe8d65bd0d6d76d5fefba2ac02f0c48a43f4.story, files number  - 3,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/002b864bf68d00d74d0cad76be4cbc049f7321ca.story, files number  - 4,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/0036d1138155e2644813f9c218c743e982e27a27.story, files number  - 5,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/0041698b4463a633f912681b96f73648cb012e33.story, files number  - 6,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/0044e296ecfe3ba57a351ad2a36d034491e878ce.story, files number  - 7,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/004b7045c61fab2a7189f8c09cae11ef1d4c5cfc.story, files number  - 8,  out of - 5607
Loading  - G:\AI\data\cnn\sam

Loading  - G:\AI\data\cnn\sample_5k\/02fbe00a8ca19dbd1ddc989bde8f387e1ea34848.story, files number  - 70,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/030c265a87f9d6364ff3299572b2dfde395adfd7.story, files number  - 71,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/0322839ca00b6659c8a3a4bdd4d36889314f9469.story, files number  - 72,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/034f46b2e694cce3e2624a4daf397cc8a3d0d012.story, files number  - 73,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/03508fefab7828edff48c0feaf8a475cc0fa2ff3.story, files number  - 74,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/0365c2945c358868792378f941fe317959df1ba3.story, files number  - 75,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/0367318ff8a01fa1527bc4e76977dd5397754110.story, files number  - 76,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/036916a23f24b5f9495e3d9c4b7014afe4cd68da.story, files number  - 77,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/036

Loading  - G:\AI\data\cnn\sample_5k\/060adbe29982b0d97da09487cc50605ac0d7cb03.story, files number  - 138,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/060eca2f7301be29235f5aabb8ca6fbb2759df60.story, files number  - 139,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/06118769addd0f4ec1c544d18bd454c23b423b1c.story, files number  - 140,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/0635f6decd89e9f5d90cbe2e8ab5ad95207fd26a.story, files number  - 141,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/0636b07ad9bd495794acf5cfde96f3abc9ab431e.story, files number  - 142,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/065ad7a8b56771c1ba6c039c5ad66311425bd42f.story, files number  - 143,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/067ea7c824ffda2bf3a1d4d5b3843f0c96c2f83a.story, files number  - 144,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/06884c5660cbe39d0d90e571f1f013ee4c446337.story, files number  - 145,  out of - 5607
Loading  - G:\AI\data\cnn\sample

Loading  - G:\AI\data\cnn\sample_5k\/08ee67a21c0fff832a920740d79dd44b07987f1c.story, files number  - 206,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/08fb1e8132b29d2ef42e35113be4715054bd1856.story, files number  - 207,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/08ffdfdcd9c6a88aa6dc00aecd5003209e0bb894.story, files number  - 208,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/090fd02416110afe7a80f9ecf1101adfdb37f373.story, files number  - 209,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/093c04d2f8b9c00d65ce5832bbd780398d8884f4.story, files number  - 210,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/094594f42aa1828294be55f844bda7cc3d52e037.story, files number  - 211,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/095060a549b7b78bb2c3610a31fd443d89f907a6.story, files number  - 212,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/0950b37d4d0c60715df19f4d2e92ddd7f0b967a9.story, files number  - 213,  out of - 5607
Loading  - G:\AI\data\cnn\sample

Loading  - G:\AI\data\cnn\sample_5k\/0bc55c4e3c651628ad0e409a8b7c15139a917e3e.story, files number  - 274,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/0bcc5c7a3e66e114aaa39dcca83d4656d5978022.story, files number  - 275,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/0be2144b436715d01b59157fbd94bdd819f259b3.story, files number  - 276,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/0be24040fa4bfa031f39bb499d73269ee8380e37.story, files number  - 277,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/0be25ec82669ce7569e0cd4446fbaa8ccd092023.story, files number  - 278,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/0bebf38305e1c46c6095ab03fffd5b925a135328.story, files number  - 279,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/0beee49fe854d6c8921dab7ebfc7fe1edda416b7.story, files number  - 280,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/0bfa55f5b55d9aaefe860a6f55cc909236a127b4.story, files number  - 281,  out of - 5607
Loading  - G:\AI\data\cnn\sample

...

Loading  - G:\AI\data\cnn\sample_5k\/f1dab6ed964a9c83e22dcd39791fa4cb472a1fad.story, files number  - 5312,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/f1deb0f1b062e57f2f85633c2b73e7329dbdd8f6.story, files number  - 5313,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/f1decbd76c1637e6bb2ecc7b7a94143970ddf101.story, files number  - 5314,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/f20460441899d8131408e13237472f57a0e50ce2.story, files number  - 5315,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/f230dc33038c2d4bd5926e69a66ad3e55e190844.story, files number  - 5316,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/f237c468e2e9cb25a09c59b8743a066890b6c6eb.story, files number  - 5317,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/f2423cc98b17ce9c4bfc6c56ad68f8b845676c5b.story, files number  - 5318,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/f25ca8ee5e5dcac2d815f79ac8af4a6975b5687c.story, files number  - 5319,  out of - 5607
Loading  - G:\AI\data\cn

Loading  - G:\AI\data\cnn\sample_5k\/f54e8334fee7d2b2057ff1d4adb2b3d8cae1b9d5.story, files number  - 5380,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/f555b81fb888354a9a83ba9acc1bdb18791441ca.story, files number  - 5381,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/f55abc14805496af56cf53489cfdb40e116567cf.story, files number  - 5382,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/f561075aa3685111dc9f4df058ed9a48ed296bc6.story, files number  - 5383,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/f56c3a0bda557e2d346a04de9682d48bbb8f874e.story, files number  - 5384,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/f56e215357fc53d2b6fa5f8aa9b4eeda86b1ad09.story, files number  - 5385,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/f56f4edb473987f48ee46681ef05028b967a875a.story, files number  - 5386,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/f5796119a1a4e977bea3d6df28f534641c1b934a.story, files number  - 5387,  out of - 5607
Loading  - G:\AI\data\cn

Loading  - G:\AI\data\cnn\sample_5k\/f7c1af4e680f7475a8da0f99e7fe75724f441a0d.story, files number  - 5448,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/f7c2eb465479b965669b81535234e24735915ebf.story, files number  - 5449,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/f7d465d04aeb023ec9fd4efcf70ffdd80ffddd17.story, files number  - 5450,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/f7d7d06a493eba6b7f1f830eeecbf01b16ddd6a3.story, files number  - 5451,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/f7dd3331408529408be2b706392e0b4a02a6a4cc.story, files number  - 5452,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/f7e0de384a8e52cafaa3161f71a979d269fe12fe.story, files number  - 5453,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/f7f13918c41ed3c6dbe208b9a71df995c2009ff4.story, files number  - 5454,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/f7f9156f26bbf12e63523bdd2f607a2f7a7f9f0c.story, files number  - 5455,  out of - 5607
Loading  - G:\AI\data\cn

Loading  - G:\AI\data\cnn\sample_5k\/fb0f8d71883d9d9413e59c78c14f0ba785b5c9a5.story, files number  - 5516,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/fb1d2b42ab656456bab0be8219e438316c88f39d.story, files number  - 5517,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/fb20a6f7c47cce73e70cdde7a3516380140b07c5.story, files number  - 5518,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/fb2b834efd122eb231037381fb0feb7533476a0e.story, files number  - 5519,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/fb310a7cc97be49363c7a49870f302fdb3f8e50f.story, files number  - 5520,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/fb466ac43f540b80312ccd8754347ce4435883b1.story, files number  - 5521,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/fb5403e103cb119efefd2e806d8e312c3d333cf7.story, files number  - 5522,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/fb54dd358d16487eca377c786af5985f82ee4d4c.story, files number  - 5523,  out of - 5607
Loading  - G:\AI\data\cn

Loading  - G:\AI\data\cnn\sample_5k\/ff08c2796456bf6b42dc8ddda12bd121c6de1437.story, files number  - 5584,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/ff08ec4366ea798b57eda92c398fba9252a773f3.story, files number  - 5585,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/ff16e4f836279ee7e300d4027cd1001d21a1cfe0.story, files number  - 5586,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/ff24845f57a9af4119607d9a9e8adf31c8178151.story, files number  - 5587,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/ff3a5618a72b644c41eb4dd9f5bef25138eaeea6.story, files number  - 5588,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/ff3d465726a4397b740e5dbb7264f988f0d08f28.story, files number  - 5589,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/ff42fad959e8b522b562b7b715c8d558270a79f2.story, files number  - 5590,  out of - 5607
Loading  - G:\AI\data\cnn\sample_5k\/ff4cf78e837bd36ead5799c8051d22a8d7b9b178.story, files number  - 5591,  out of - 5607
Loading  - G:\AI\data\cn